In [1]:
import gc, time, pandas as pd, numpy as np

VERBOSE = True          # master switch for debug prints
PRINT_EVERY = 10        # print every N matches
SHOW_HEAD = 0           # set to >0 to show first N rows of each slice

def _mem_mb(obj=None):
    try:
        if obj is None:
            import os, psutil
            process = psutil.Process(os.getpid())
            return process.memory_info().rss / 1e6
        else:
            return obj.memory_usage(deep=True).sum() / 1e6
    except Exception:
        # Fallback if psutil isn't available
        return None

In [2]:
df = pd.read_csv("All_events_labeled.csv")

keep = [
     "match_id","team_id","possession","possession_team_id",
    "minute","second","x","y","end_x","end_y",
    "type","shot_statsbomb_xg",
    "pass_end_x","pass_end_y","carry_end_x","carry_end_y",
    "index"  # tie-breaker for ordering
]

df = df[keep].copy()



C:\Users\Areen\AppData\Local\Temp\ipykernel_18052\2371259621.py:1: DtypeWarning: Columns (0,2,3,4,6,7,8,9,10,11,13,14,15,16,17,19,20,21,22,23,24,25,26,27,28,29,30,31,34,39,45,46,48,50,53,57,59,60,72,73,74,75,76,77,78,80,81,82,84,86,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("All_events_labeled.csv")


In [3]:
# integers
for c in ["match_id","team_id","possession","possession_team_id","minute","second","index"]:
    if c in df:
        df[c] = pd.to_numeric(df[c], errors="coerce").astype("Int32")  # pandas nullable, compact

# floats
for c in ["x","y","end_x","end_y","pass_end_x","pass_end_y","carry_end_x","carry_end_y","shot_statsbomb_xg"]:
    if c in df:
        df[c] = pd.to_numeric(df[c], errors="coerce").astype("float32")

# strings → categoricals
for c in ["type"]:
    if c in df:
        df[c] = df[c].astype("category")

# derived time
df["abs_second"] = (df["minute"].fillna(0).astype("float32")*60 + df["second"].fillna(0).astype("float32")).astype("float32")

gc.collect()
print(df.info(memory_usage="deep"))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2865562 entries, 0 to 2865561
Data columns (total 18 columns):
 #   Column              Dtype   
---  ------              -----   
 0   match_id            Int32   
 1   team_id             Int32   
 2   possession          Int32   
 3   possession_team_id  Int32   
 4   minute              Int32   
 5   second              Int32   
 6   x                   float32 
 7   y                   float32 
 8   end_x               float32 
 9   end_y               float32 
 10  type                category
 11  shot_statsbomb_xg   float32 
 12  pass_end_x          float32 
 13  pass_end_y          float32 
 14  carry_end_x         float32 
 15  carry_end_y         float32 
 16  index               Int32   
 17  abs_second          float32 
dtypes: Int32(7), category(1), float32(10)
memory usage: 207.7 MB
None


In [4]:
# Only touch rows that are NaN to avoid copies
mask_ex = df["end_x"].isna() & df["pass_end_x"].notna()
# Creates a boolean mask for rows where end_x is missing and pass_end_x exists.
# Goal: identify the safest places to fill end_x from the pass endpoint.

df.loc[mask_ex, "end_x"] = df.loc[mask_ex, "pass_end_x"].astype("float32")
# Fills end_x with pass_end_x only on those masked rows.
# Casts to float32 to save memory and keep a consistent dtype.

mask_ey = df["end_y"].isna() & df["pass_end_y"].notna()
df.loc[mask_ey, "end_y"] = df.loc[mask_ey, "pass_end_y"].astype("float32")
# Same idea, but for the Y coordinate: fill missing end_y from pass_end_y.

mask_ex2 = df["end_x"].isna() & df["carry_end_x"].notna()
df.loc[mask_ex2, "end_x"] = df.loc[mask_ex2, "carry_end_x"].astype("float32")
mask_ey2 = df["end_y"].isna() & df["carry_end_y"].notna()
df.loc[mask_ey2, "end_y"] = df.loc[mask_ey2, "carry_end_y"].astype("float32")
# For any remaining missing end_x/end_y, try to fill from the carry endpoint (carry_end_x, carry_end_y).
# Order matters: you first try pass endpoints, then fall back to carry endpoints.

df["end_x"] = df["end_x"].fillna(df["x"]).astype("float32")
df["end_y"] = df["end_y"].fillna(df["y"]).astype("float32")
# Final fallback: if end_x/end_y are still missing, use the start location (x, y) so there are no NaNs left.
# Cast again to float32 to ensure the column stays compact.

In [5]:
# 1) Build a boolean "is_shot" efficiently, even on huge data
start_all = time.perf_counter()


is_shot = (

    df["type"].cat.codes == df["type"].cat.categories.get_indexer(["Shot"])[0]
    
    ) if "Shot" in list(df["type"].cat.categories) else df["type"].astype(str).eq("Shot")
#Creates a boolean series that is True when the event type is a Shot and false otherwise


df["shot_xg"] = np.where(is_shot, df["shot_statsbomb_xg"].fillna(0).astype("float32"), np.float32(0.0))
# 


In [6]:
# --- 1) Stream by match -------------------------------------------------------
poss_parts = []
n_matches = df["match_id"].nunique()
t0 = time.perf_counter()

keys = ["possession","possession_team_id"]

for i, (mid, g) in enumerate(df.groupby("match_id", sort=False, observed=True), start=1):
    t_match = time.perf_counter()
    if VERBOSE and (i == 1 or i % PRINT_EVERY == 0):
        print(f"[MATCH {i}/{n_matches}] match_id={mid} | rows={len(g)} | df_mem={_mem_mb(df):.1f} MB | rss={_mem_mb():.1f} MB")
        if SHOW_HEAD:
            print(g.head(SHOW_HEAD))

        # rows that actually have start locations
    g_startable = g[g["x"].notna() & g["y"].notna()]
    starts = g_startable.groupby(keys, observed=True).agg(
        start_x=("x","first"),
        start_y=("y","first"),
        start_minute=("minute","first"),
        start_second=("second","first"),
    )

    # rows that actually have end locations
    g_endable = g[g["end_x"].notna() & g["end_y"].notna()]
    ends = g_endable.groupby(keys, observed=True).agg(
        end_x=("end_x","last"),
        end_y=("end_y","last"),
        end_minute=("minute","last"),
        end_second=("second","last"),
)

    # duration (in seconds) using any rows that have time
    dur = g.groupby(keys, observed=True)["abs_second"] \
       .agg(lambda s: (s.max() - s.min()) if s.notna().any() else np.nan) \
       .rename("duration_s")
    
    # base counts (same as before)
    grp = g.groupby(keys, sort=False, observed=True)
    base = grp.agg(
        n_events=("type","size"),
        n_shots=("shot_xg", lambda s: (s > 0).sum()),
    )

    # target and progression (same as before)
    labels = grp["shot_xg"].sum().astype("float32").rename("possession_xg")
    dx = (g["end_x"] - g["x"]).astype("float32")
    g_dx = g.assign(dx=dx)
    prog = g_dx.groupby(keys, observed=True)["dx"] \
            .apply(lambda s: s.clip(lower=0).sum().astype("float32")) \
            .rename("prog_dist")

    # Basic features
    base = grp.agg(
        n_events=("type","size"),
        n_shots=("shot_xg", lambda s: (s > 0).sum()),
        start_x=("x","first"), start_y=("y","first"),
        end_x=("end_x","last"), end_y=("end_y","last"),
        start_minute=("minute","first"), start_second=("second","first"),
        end_minute=("minute","last"),   end_second=("second","last"),
    )

    poss_mid = (base.join(labels, how="left")
                    .join(prog,   how="left")
                    .reset_index())
    poss_mid.insert(0, "match_id", mid)

    # Debug: show small summary for this match
    if VERBOSE and (i == 1 or i % PRINT_EVERY == 0):
        print(f"    [+] poss rows: {len(poss_mid)} | "
              f"mean_xg={float(poss_mid['possession_xg'].mean()):.4f} | "
              f"pct_high={(poss_mid['possession_xg'] >= 0.15).mean():.3f} | "
              f"elapsed={time.perf_counter() - t_match:.2f}s")

    poss_parts.append(poss_mid)

    # Free per-match intermediates
    del g, grp, poss_mid, base, labels, prog, g_dx, dx
    if i % PRINT_EVERY == 0:
        gc.collect()
        if VERBOSE:
            print(f"    [GC] rss after GC ≈ {_mem_mb():.1f} MB\n")

# --- 2) Combine & finalize ----------------------------------------------------
poss = pd.concat(poss_parts, ignore_index=True)
poss["possession_xg"] = poss["possession_xg"].fillna(0).astype("float32")
poss["high_xg"] = (poss["possession_xg"] >= 0.15).astype("int8")

del poss_parts
gc.collect()

if VERBOSE:
    total_s = time.perf_counter() - start_all
    print(f"\n[DONE] poss shape: {poss.shape} | poss mem: {_mem_mb(poss):.1f} MB | total time: {total_s:.2f}s")

# Optional: quick sanity print
print(poss.head(3))

# How many possessions still have missing start/end coords?
missing_start = poss["start_x"].isna().mean()
missing_end   = poss["end_x"].isna().mean()
print(f"% missing start: {missing_start:.3%} | % missing end: {missing_end:.3%}")

# Progression can exceed 120 because it sums all forward segments in the possession.
# If you prefer a cap, use straight-line net progress instead:
poss["net_prog"] = (poss["end_x"] - poss["start_x"]).clip(lower=0).astype("float32")


[MATCH 1/817] match_id=22912 | rows=3165 | df_mem=229.2 MB | rss=871.6 MB
    [+] poss rows: 172 | mean_xg=0.0124 | pct_high=0.012 | elapsed=0.15s
[MATCH 10/817] match_id=18235 | rows=3410 | df_mem=229.2 MB | rss=776.1 MB
    [+] poss rows: 188 | mean_xg=0.0119 | pct_high=0.027 | elapsed=0.12s
    [GC] rss after GC ≈ 775.3 MB

[MATCH 20/817] match_id=3754245 | rows=3078 | df_mem=229.2 MB | rss=758.2 MB
    [+] poss rows: 168 | mean_xg=0.0128 | pct_high=0.024 | elapsed=0.11s
    [GC] rss after GC ≈ 758.4 MB

[MATCH 30/817] match_id=3754050 | rows=3047 | df_mem=229.2 MB | rss=759.4 MB
    [+] poss rows: 198 | mean_xg=0.0110 | pct_high=0.010 | elapsed=0.13s
    [GC] rss after GC ≈ 759.6 MB

[MATCH 40/817] match_id=3754146 | rows=3316 | df_mem=229.2 MB | rss=760.0 MB
    [+] poss rows: 200 | mean_xg=0.0147 | pct_high=0.020 | elapsed=0.12s
    [GC] rss after GC ≈ 760.0 MB

[MATCH 50/817] match_id=3754015 | rows=3645 | df_mem=229.2 MB | rss=760.1 MB
    [+] poss rows: 188 | mean_xg=0.0036 | 

In [7]:
import pandas as pd
import numpy as np
import time, gc, os

# ---- Inputs ----
ev = df.copy()            # your full events DataFrame
ps = poss.copy()          # your possession table (has start/end times)

# ---- 1) Clean basic fields & coordinates ----
for c in ["minute","second"]:
    ev[c] = pd.to_numeric(ev.get(c), errors="coerce").fillna(0)
ev["abs_second"] = ev["minute"]*60 + ev["second"]

# end_x/end_y: fill from pass/carry, else fallback to start
for a, b in [("end_x","x"), ("end_y","y")]:
    if a not in ev.columns:
        ev[a] = np.nan
        
if {"pass_end_x","pass_end_y"}.issubset(ev.columns):

    m = ev["end_x"].isna() & ev["pass_end_x"].notna()
    ev.loc[m, "end_x"] = pd.to_numeric(ev.loc[m,"pass_end_x"], errors="coerce")
    m = ev["end_y"].isna() & ev["pass_end_y"].notna()
    ev.loc[m, "end_y"] = pd.to_numeric(ev.loc[m,"pass_end_y"], errors="coerce")

if {"carry_end_x","carry_end_y"}.issubset(ev.columns):

    m = ev["end_x"].isna() & ev["carry_end_x"].notna()
    ev.loc[m, "end_x"] = pd.to_numeric(ev.loc[m,"carry_end_x"], errors="coerce")
    m = ev["end_y"].isna() & ev["carry_end_y"].notna()
    ev.loc[m, "end_y"] = pd.to_numeric(ev.loc[m,"carry_end_y"], errors="coerce")

ev["end_x"] = ev["end_x"].fillna(ev.get("x"))
ev["end_y"] = ev["end_y"].fillna(ev.get("y"))

# ensure 'index' exists (stable within match)
if "index" not in ev.columns:
    ev = ev.sort_values(["match_id","minute","second"]).reset_index(drop=True)
    ev["index"] = ev.groupby("match_id").cumcount()

# ---- 2) Ensure model/EDA columns exist ----
if "player_id" not in ev.columns:
    if "player" in ev.columns:
        ev["player_id"] = ev["player"].astype("category").cat.codes.astype("Int32")
    else:
        ev["player_id"] = pd.Series(pd.NA, index=ev.index, dtype="Int32")
if "shot_outcome" not in ev.columns:
    ev["shot_outcome"] = pd.Series(pd.NA, index=ev.index, dtype="string")
if "pass_length" not in ev.columns:
    if {"x","y","pass_end_x","pass_end_y"}.issubset(ev.columns):
        dx = pd.to_numeric(ev["pass_end_x"], errors="coerce") - pd.to_numeric(ev["x"], errors="coerce")
        dy = pd.to_numeric(ev["pass_end_y"], errors="coerce") - pd.to_numeric(ev["y"], errors="coerce")
        ev["pass_length"] = np.sqrt(dx**2 + dy**2)
    else:
        ev["pass_length"] = np.nan
if "pass_outcome" not in ev.columns:
    ev["pass_outcome"] = pd.Series(pd.NA, index=ev.index, dtype="string")
if "shot_statsbomb_xg" not in ev.columns:
    ev["shot_statsbomb_xg"] = 0.0
if "type" not in ev.columns:
    ev["type"] = ""

# ---- 3) (RE)BUILD possession from poss if missing ----
if "possession" not in ev.columns or ev["possession"].isna().all():
    ps = ps.copy()
    for c in ["start_minute","start_second","end_minute","end_second"]:
        ps[c] = pd.to_numeric(ps[c], errors="coerce").fillna(0)
    ps["start_s"] = ps["start_minute"]*60 + ps["start_second"]
    ps["end_s"]   = ps["end_minute"]*60   + ps["end_second"]

    ev["possession"] = pd.Series(pd.NA, index=ev.index, dtype="Int32")
    ev["possession_team_id_from_poss"] = pd.Series(pd.NA, index=ev.index, dtype="Int32")

    t0 = time.perf_counter()
    for i, (mid, g_poss) in enumerate(ps.groupby("match_id", observed=True), 1):
        idx_ev = ev.index[ev["match_id"] == mid]
        if len(idx_ev) == 0: 
            continue
        g_ev = ev.loc[idx_ev, ["abs_second"]]

        intervals = pd.IntervalIndex.from_arrays(g_poss["start_s"], g_poss["end_s"], closed="both")
        hit = intervals.get_indexer(g_ev["abs_second"])
        take = hit != -1
        ev.loc[idx_ev[take], "possession"] = g_poss["possession"].to_numpy()[hit[take]]
        ev.loc[idx_ev[take], "possession_team_id_from_poss"] = g_poss["possession_team_id"].to_numpy()[hit[take]]

        if i % 50 == 0:
            print(f"[assign] match {mid}: events={len(idx_ev)} assigned={take.sum()}")
        del g_ev, intervals, hit, take
        gc.collect()
    print(f"[assign] done in {time.perf_counter()-t0:.1f}s; assigned {(ev['possession'].notna().mean()*100):.2f}% of events")

# ---- 4) Build the export with everything you need later ----
keep_cols = [
    "match_id","possession","team_id","player_id","type",
    "x","y","end_x","end_y","minute","second","index",
    "shot_statsbomb_xg","shot_outcome","pass_length","pass_outcome"
]
keep_cols = [c for c in keep_cols if c in ev.columns]
events_compact = ev[keep_cols].copy()

# ---- 5) Save CSVs ----
events_csv = r"C:\Users\Areen\StatsBomb_value_actions\events_compact.csv"
events_compact.to_csv(events_csv, index=False)
print(f"Saved {events_csv} | shape={events_compact.shape}")

poss_csv = r"C:\Users\Areen\StatsBomb_value_actions\possessions.csv"
ps_out = poss.copy()
ps_out.to_csv(poss_csv, index=False)
print(f"Saved {poss_csv} | shape={ps_out.shape}")


Saved C:\Users\Areen\StatsBomb_value_actions\events_compact.csv | shape=(2865562, 16)
Saved C:\Users\Areen\StatsBomb_value_actions\possessions.csv | shape=(160009, 17)


In [8]:
print(events_compact)

print(poss)

         match_id  possession  team_id  player_id          type   x   y  \
0           22912           1       38       <NA>   Starting XI NaN NaN   
1           22912           1       24       <NA>   Starting XI NaN NaN   
2           22912           1       24       <NA>    Half Start NaN NaN   
3           22912           1       38       <NA>    Half Start NaN NaN   
4           22912          81       24       <NA>    Half Start NaN NaN   
...           ...         ...      ...        ...           ...  ..  ..   
2865557   3889182         227      227       <NA>      Half End NaN NaN   
2865558   3889182         129      227       <NA>  Substitution NaN NaN   
2865559   3889182         140      224       <NA>  Substitution NaN NaN   
2865560   3889182         197      224       <NA>  Substitution NaN NaN   
2865561   3889182         197      227       <NA>  Substitution NaN NaN   

         end_x  end_y  minute  second  index  shot_statsbomb_xg shot_outcome  \
0          NaN    N

Project data overview
Source / grain
Events table (events): one row = one on-ball event from StatsBomb-style data.

Possessions table (poss): one row = one possession (as defined by StatsBomb’s possession within each match_id).
1) Events table (events / trimmed copy df) Core columns we used (among ~126 available)

| Column               | Type         | Unit / Domain                  | Meaning                                                                                      |
| -------------------- | ------------ | ------------------------------ | -------------------------------------------------------------------------------------------- |
| `match_id`           | Int          | id                             | Match identifier.                                                                            |
| `team_id`            | Int          | id                             | Team executing the event.                                                                    |
| `possession`         | Int          | id (per match)                 | StatsBomb possession id (resets each match).                                                 |
| `possession_team_id` | Int          | id                             | Team credited with the possession (can differ from `team_id` on certain events).             |
| `minute`, `second`   | Int          | minutes, seconds               | Event clock.                                                                                 |
| `timestamp`          | str/time     | ISO-like                       | Original timestamp (not required for modeling).                                              |
| `x`, `y`             | float32      | pitch coords (0–120, 0–80)     | Event start location (StatsBomb scale).                                                      |
| `end_x`, `end_y`     | float32      | pitch coords                   | Event end location (filled from `pass_*`/`carry_*` if missing, otherwise fallback to start). |
| `type`               | category/str | e.g., “Pass”, “Carry”, “Shot”… | Event type label.                                                                            |
| `shot_statsbomb_xg`  | float        | 0–1                            | xG for shot events; NaN otherwise.                                                           |
| `abs_second`         | float32      | seconds                        | Derived: `minute*60 + second`.                                                               |
| `shot_xg`            | float32      | 0–1                            | Derived: `shot_statsbomb_xg` for Shot rows; `0.0` otherwise.                                 |

Important derived/cleaning rules

    Filled end_x/end_y from pass_end_x/pass_end_y then carry_end_x/carry_end_y, else fallback to x/y.

    shot_xg = shot_statsbomb_xg for rows where type == "Shot" (else 0.0).

    Sorted by match_id, (period if present), abs_second, index (stable).

2) Possessions table (poss)
Built by streaming per match_id to control memory.

| Column                         | Type    | Unit / Domain  | Meaning                                                                                                            |
| ------------------------------ | ------- | -------------- | ------------------------------------------------------------------------------------------------------------------ |
| `match_id`                     | Int     | id             | Match identifier.                                                                                                  |
| `possession`                   | Int     | id (per match) | Possession id (from events).                                                                                       |
| `possession_team_id`           | Int     | id             | Team owning the possession.                                                                                        |
| `n_events`                     | Int     | count          | # of events in the possession.                                                                                     |
| `n_shots`                      | Int     | count          | # of shots in the possession (based on `shot_xg > 0`).                                                             |
| `possession_xg`                | float32 | 0–?            | **Target**: sum of `shot_xg` within the possession.                                                                |
| `high_xg`                      | int8    | 0/1            | Optional label: `1` if `possession_xg ≥ 0.15`, else `0`.                                                           |
| `prog_dist`                    | float32 | length units   | Sum of **positive** `(end_x − x)` across all events in the possession (accumulates forward moves; can exceed 120). |
| `start_x`, `start_y`           | float32 | pitch coords   | First valid start location in the possession.                                                                      |
| `end_x`, `end_y`               | float32 | pitch coords   | Last valid end location in the possession.                                                                         |
| `start_minute`, `start_second` | Int     | time           | First valid event time in the possession.                                                                          |
| `end_minute`, `end_second`     | Int     | time           | Last valid event time in the possession.                                                                           |
| *(optional)* `duration_s`      | float32 | seconds        | `max(abs_second) − min(abs_second)` per possession (if added).                                                     |
| *(optional)* `net_prog`        | float32 | length units   | `(end_x − start_x).clip(lower=0)`; bounded progression proxy.                                                      |


Target definition (for the baseline)
Regression target: possession_xg = Σ shot_xg within the possession.

    Rationale: continuous value gives richer signal; you can threshold later for “high-xG”.

Optional binary label: high_xg = 1 if possession_xg ≥ 0.15, else 0.

Intended features for the baseline model
(We’ll start simple; can expand later.)

From poss (already computed):
    Counts: n_events, n_shots
    Progression: prog_dist (and/or net_prog)
    Geometry: start_x, start_y, end_x, end_y
    Timing: start_minute/second, end_minute/second (or duration_s if added)
    IDs for grouping/splitting: match_id, possession, possession_team_id (not used as predictors)
    From events (easy future adds if desired):
    Pass count/success, dribble count/success, entries into final third/box, average action angle/distance, etc.


Splitting & evaluation plan
    Split by match_id (GroupKFold or time-based) to avoid leakage.
    Metrics (regression): MAE, RMSE, R², plus calibration (bin predicted xG and compare to realized).
    Team/match sanity: per-match sum of predicted possession xG vs. actual sum

Caveats / assumptions
    We trust StatsBomb’s possession segmentation.
    prog_dist is cumulative forward movement; zig-zags can inflate it. net_prog offers a bounded alternative.
    A small fraction of possessions have no locational events → remaining NaNs possible on start/end fields.
